### Import dataset

In [1]:
import pandas as pd
import numpy as np

In [2]:
gas = pd.read_csv('Natural_Gas_Consumption_by_ZIP_Code_-_2010_clean.csv', sep = ',')
gas.head()

,Zip Code,Building type (service class),Consumption (therms),Utility/Data Source,Latitude,Longitude
0,11109,Commercial,45899.0,ConEd,40.744415,-73.957702
1,11429,Commercial,755.0,ConEd,40.709913,-73.738640
2,11226,Industrial,65835.0,National Grid,40.646505,-73.957190
3,10314,Institutional,2294516.0,National Grid,40.596490,-74.165991
4,11223,Commercial,2376036.0,National Grid,40.596940,-73.973311


In [3]:
gas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 964 entries, 0 to 963
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Zip Code                       964 non-null    int64  
 1   Building type (service class)  964 non-null    object 
 2   Consumption (therms)           964 non-null    float64
 3   Utility/Data Source            964 non-null    object 
 4   Latitude                       964 non-null    float64
 5   Longitude                      964 non-null    float64
dtypes: float64(3), int64(1), object(2)
memory usage: 45.3+ KB


## Import folium package and map types.<br/><br/> We will import Choropleth, HeatMap, Marker, MarkerCluster, and Circle Maps

In [4]:
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

## Marker Map

### Display blank basemap using folium

In [13]:
map1 = folium.Map(location = [np.mean(gas['Latitude']), np.mean(gas['Longitude'])], tiles = 'OpenStreetMap', 
                  zoom_start = 10)

folium.TileLayer('Stamen Terrain').add_to(map1)
folium.TileLayer('cartodbdark_matter').add_to(map1)
folium.TileLayer('Stamen Water Color').add_to(map1)
folium.TileLayer('Stamen Toner').add_to(map1)
folium.TileLayer('cartodbpositron').add_to(map1)

folium.LayerControl().add_to(map1)
map1

### Add markers to basemap using Marker in Folium using folium.Marker() and a loop for each marker. <br/> <br/>Each marker represents a different gas account

In [11]:
for idx, row in gas.iterrows():
    Marker([row['Latitude'], row['Longitude']]).add_to(map1)
map1

### There are too many markers to get a good idea of where our accounts are concentrated.<br/><br/> With a MarkerCluster Map we can get a better idea of where our accounts are located since accounts in close proximity to one another are grouped together.

## MarkerCluster Map

In [12]:
import math 

map2 = folium.Map(location = [np.mean(gas['Latitude']), np.mean(gas['Longitude'])], tiles = 'OpenStreetMap', 
                  zoom_start = 8)

mc = MarkerCluster(name = 'Clustered Accounts')

for idx, row in gas.iterrows():
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(Marker([row['Latitude'], row['Longitude']]))
        
map2.add_child(mc)

map2

### If we go back to the dataset we can see that the gas accounts are either coming from National Grid or ConEd. How can we visualize our National Grid accounts seperate from our ConEd accounts.<br/><br/>One way to do this is by creating a HeatMap

In [8]:
gas.head()

,Zip Code,Building type (service class),Consumption (therms),Utility/Data Source,Latitude,Longitude
0,11109,Commercial,45899.0,ConEd,40.744415,-73.957702
1,11429,Commercial,755.0,ConEd,40.709913,-73.738640
2,11226,Industrial,65835.0,National Grid,40.646505,-73.957190
3,10314,Institutional,2294516.0,National Grid,40.596490,-74.165991
4,11223,Commercial,2376036.0,National Grid,40.596940,-73.973311


In [9]:
gas['Utility/Data Source'].value_counts()

ConEd            518
National Grid    446
Name: Utility/Data Source, dtype: int64

## HeatMap

In [10]:
map3 = folium.Map(location = [np.mean(gas['Latitude']), np.mean(gas['Longitude'])], tiles = 'OpenStreetMap',
                             zoom_start = 10)

HeatMap(gas[gas['Utility/Data Source'] == 'National Grid'][['Latitude', 'Longitude']], radius = 10,
       gradient = {0.5: 'purple', .65: 'gold', 1: 'white'}, name = 'National Grid').add_to(map3)

HeatMap(gas[gas['Utility/Data Source'] == 'ConEd'][['Latitude', 'Longitude']], radius = 10, 
       gradient = {0.5: 'black', .65: 'red', 1: 'yellow'}, name = 'ConEd').add_to(map3)

folium.LayerControl().add_to(map3)

map3

## Bubble Map

### If we go back again to the to the dataset we can see that the account types are in the Building (service class) column. 

In [11]:
gas.head()

,Zip Code,Building type (service class),Consumption (therms),Utility/Data Source,Latitude,Longitude
0,11109,Commercial,45899.0,ConEd,40.744415,-73.957702
1,11429,Commercial,755.0,ConEd,40.709913,-73.738640
2,11226,Industrial,65835.0,National Grid,40.646505,-73.957190
3,10314,Institutional,2294516.0,National Grid,40.596490,-74.165991
4,11223,Commercial,2376036.0,National Grid,40.596940,-73.973311


In [12]:
gas['Building type (service class)'].value_counts()

Commercial           338
Residential          198
Large residential    165
Small residential     96
Institutional         86
Industrial            81
Name: Building type (service class), dtype: int64

### How could we visualize each of these account types using a Bubble Map?

In [13]:
# Define a new map

map4 = folium.Map(location = [np.mean(gas['Latitude']), np.mean(gas['Longitude'])], tiles = 'OpenStreetMap', 
                  zoom_start = 10)

# Color function 

def bubble_color(val):
    
    if val == 'Commercial':
        return 'red'
    
    elif val == 'Residential':
        return 'yellow'
    
    elif val == 'Large residential':
        return 'blue'
    
    elif val == 'Small residential':
        return 'green'
    
    elif val == 'Institutional':
        return 'purple'
    
    else:
        return 'orange'


# Add Bubble layer to map using folium Circle function

for i in range(len(gas)):
    Circle(location = [gas.iloc[i]['Latitude'], gas.iloc[i]['Longitude']], 
          radius = 20, color = bubble_color(gas['Building type (service class)'][i])).add_to(map4)

folium.LayerControl().add_to(map4)

# Display map

map4

## Choropleth Maps

### Choropleth maps allow for pre-defined areas to be colored or patterned in proportion to a statisitical variable. For example, you could check for average housing values by state in a map where darker areas would have higher prices.<br/> <br/>Let's say we wanted to know which borough uses the most and which one uses the least.<br/><br/>We could figure this out with a spatial join between the borough shapefiles and 

### import packages

In [14]:
import geopandas as gpd # Pandas GeodDataFrame
from geopandas.tools import sjoin # Join shapefiles to coordinate data
from shapely.geometry import Point # Geopoints

### Create a GeoSeries to join latitudes and longitudes as points

In [15]:
points = gpd.GeoSeries(gas.apply(lambda x: Point(x['Longitude'], x['Latitude']), 1), crs = {'init': 'epsg: 4326'})
points.head()

0    POINT (-73.95770 40.74441)
1    POINT (-73.73864 40.70991)
2    POINT (-73.95719 40.64651)
3    POINT (-74.16599 40.59649)
4    POINT (-73.97331 40.59694)
dtype: geometry

In [16]:
gas_gdf = gpd.GeoDataFrame(gas.drop(['Latitude', 'Longitude'], axis = 1) , geometry = points)
gas_gdf.head()

,Zip Code,Building type (service class),Consumption (therms),Utility/Data Source,geometry
0,11109,Commercial,45899.0,ConEd,POINT (-73.95770 40.74441)
1,11429,Commercial,755.0,ConEd,POINT (-73.73864 40.70991)
2,11226,Industrial,65835.0,National Grid,POINT (-73.95719 40.64651)
3,10314,Institutional,2294516.0,National Grid,POINT (-74.16599 40.59649)
4,11223,Commercial,2376036.0,National Grid,POINT (-73.97331 40.59694)


### We want to add the Neighborhood Tabulation Areas to our Map, but we need to extract them from a zipfile using the zipfile package

### Get shapefiles from zip folder

In [17]:
import zipfile

with zipfile.ZipFile('files/Borough Boundaries (Water Areas Included).zip', 'r') as my_zip:
    print(my_zip.namelist())
    my_zip.extractall('files')

['geo_export_49512982-8e61-4a09-91ce-6a6640b8d43c.dbf', 'geo_export_49512982-8e61-4a09-91ce-6a6640b8d43c.shp', 'geo_export_49512982-8e61-4a09-91ce-6a6640b8d43c.shx', 'geo_export_49512982-8e61-4a09-91ce-6a6640b8d43c.prj']


### Store filenames from zip folder as variables

In [18]:
dbf, shp, shx, prj = my_zip.namelist()

local_path = 'files/'

In [19]:
boroughs = gpd.read_file(local_path + shp)
boroughs.head()

,boro_code,boro_name,shape_area,shape_leng,geometry
0,1.0,Manhattan,9.442947e+08,203803.216852,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ..."
1,2.0,Bronx,1.598380e+09,188054.198841,"POLYGON ((-73.86477 40.90201, -73.86305 40.901..."
2,3.0,Brooklyn,2.684411e+09,234928.658563,"POLYGON ((-73.92722 40.72533, -73.92654 40.724..."
3,4.0,Queens,3.858050e+09,429586.630985,"POLYGON ((-73.77896 40.81171, -73.76371 40.793..."
4,5.0,Staten Island,2.539686e+09,212213.139971,"POLYGON ((-74.05581 40.64971, -74.05619 40.639..."


### Make sure boroughs shapefile and GeoDataFrame have same CRS

In [20]:
# Check crs for gas accounts

gas_gdf.crs

{'init': 'epsg: 4326'}

In [21]:
# Check crs  for boroughs shapefile

boroughs.crs

{'init': 'epsg:4326'}

In [22]:
boroughs.crs = {'init': 'epsg: 4326'}

### Join our gas accounts to the boroughs shapefile using an inner spatial join

In [23]:
# sum join

gas_sum_join = gpd.sjoin(gas_gdf, boroughs, how = 'inner').groupby('boro_name').sum()

# mean join

gas_mean_join = gpd.sjoin(gas_gdf, boroughs, how = 'inner').groupby('boro_name').mean()

# count join

gas_counts_join = gpd.sjoin(gas_gdf, boroughs, how = 'inner').groupby('boro_name').count()

In [24]:
gas_sum_join['Consumption (therms)']

boro_name
Bronx            343565702.0
Brooklyn         971108836.0
Manhattan        511669216.0
Queens           743072286.0
Staten Island    196394664.0
Name: Consumption (therms), dtype: float64

In [25]:
gas_mean_join['Consumption (therms)']

boro_name
Bronx            2.663300e+06
Brooklyn         5.005716e+06
Manhattan        2.244163e+06
Queens           2.172726e+06
Staten Island    3.167656e+06
Name: Consumption (therms), dtype: float64

In [26]:
boroughs

,boro_code,boro_name,shape_area,shape_leng,geometry
0,1.0,Manhattan,9.442947e+08,203803.216852,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ..."
1,2.0,Bronx,1.598380e+09,188054.198841,"POLYGON ((-73.86477 40.90201, -73.86305 40.901..."
2,3.0,Brooklyn,2.684411e+09,234928.658563,"POLYGON ((-73.92722 40.72533, -73.92654 40.724..."
3,4.0,Queens,3.858050e+09,429586.630985,"POLYGON ((-73.77896 40.81171, -73.76371 40.793..."
4,5.0,Staten Island,2.539686e+09,212213.139971,"POLYGON ((-74.05581 40.64971, -74.05619 40.639..."


In [27]:
# Define new map 

map5 = folium.Map(location = [np.mean(gas['Latitude']), np.mean(gas['Longitude'])], tiles = 'OpenStreetMap',
                  zoom_start = 10)

# Add Choropleth Layer

Choropleth(geo_data = boroughs.__geo_interface__, data = gas_mean_join['Consumption (therms)'], 
          key_on = 'feature.properties.boro_name', fill_color = 'BuPu', legend_name = 'Average Therms Per Borough', 
          name = 'Borough Shapefiles').add_to(map5)

# Add Layer control

folium.LayerControl().add_to(map5)

# Display map

map5

## Combination Map

In [28]:
# Define new map 

map6 = folium.Map(location = [np.mean(gas['Latitude']), np.mean(gas['Longitude'])], tiles = 'OpenStreetMap',
                  zoom_start = 10)

# Add Choropleth Layer

Choropleth(geo_data = boroughs.__geo_interface__, data = gas_sum_join['Consumption (therms)'], 
          key_on = 'feature.properties.boro_name', fill_color = 'RdBu', legend_name = 'Average Therms Per Borough', 
          name = 'Borough Shapefiles').add_to(map6)

# Add MarkerCluster Layer

mc2 = MarkerCluster(name = 'Clustered Accounts')

for idx, row in gas.iterrows():
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc2.add_child(Marker([row['Latitude'], row['Longitude']]))
    
map6.add_child(mc2)

# Add Layer control

folium.LayerControl().add_to(map6)

# Display map

map6

In [29]:
import pipreqs

In [31]:
pipreqs?